In [ ]:
from kaggle_environments import make
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col

import numpy as np

In [ ]:
def geese_heads(obs_dict, config_dict):
    """
    Return the position of the geese's heads
    """
    configuration = Configuration(config_dict)

    observation = Observation(obs_dict)
    player_index = observation.index
    player_goose = observation.geese[player_index]
    player_head = player_goose[0]
    player_row, player_column = row_col(player_head, configuration.columns)
    positions = []
    for geese in observation.geese:
        if len(geese)>0:
            geese_head = geese[0]
            row, column = row_col(geese_head, configuration.columns)
        else:
            row = None
            column = None
        positions.append((row, column))
    return positions

def get_last_actions(previous_geese_heads, heads_positions):

    def get_last_action(prev, cur):
        last_action = None

        prev_row = prev[0]
        prev_col = prev[1]
        cur_row = cur[0]
        cur_col = cur[1]

        if cur_row is not None:
            if (cur_row-prev_row == 1) | ((cur_row==0) & (prev_row==6)):
                last_action = Action.SOUTH.name
            elif (cur_row-prev_row == -1) | ((cur_row==6) & (prev_row==0)):
                last_action = Action.NORTH.name
            elif (cur_col-prev_col == 1) | ((cur_col==0) & (prev_col==10)):
                last_action = Action.EAST.name
            elif (cur_col-prev_col == -1) | ((cur_col==10) & (prev_col==0)):
                last_action = Action.WEST.name

        return last_action

    if len(previous_geese_heads) == 0:
        actions = [Action.SOUTH.name, Action.NORTH.name, Action.EAST.name, Action.WEST.name]
        nb_geeses = len(heads_positions)
        last_actions = ["None" for _ in range(nb_geeses)]
    else:   
        last_actions = [get_last_action(*pos) for pos in zip(previous_geese_heads, heads_positions)]

    return last_actions
    
def central_state_space(obs_dict, config_dict, last_actions):
    """
    Recreating a board where my agent's head in the middle of the board 
    (position (4,5)), and creating features accordingly
    """
    
    last_actions_dict = {
        Action.WEST.name: 1,
        Action.EAST.name: 2,
        Action.NORTH.name: 3,
        Action.SOUTH.name: 4,
        "None": 16
    }
    
    configuration = Configuration(config_dict)

    observation = Observation(obs_dict)
    player_index = observation.index
    player_goose = observation.geese[player_index]
    player_head = player_goose[0]
    longuest_opponent = 0
    for i, goose in enumerate(observation.geese):
        if i != player_index:
            opponent_length = len(goose)
            if opponent_length > longuest_opponent:
                longuest_opponent = opponent_length
    player_row, player_column = row_col(player_head, configuration.columns)
    row_offset = player_row - 3
    column_offset = player_row - 5

    foods = observation['food']

    def centralize(row, col):
        if col > player_column:
            new_col = (5 + col - player_column) % 11
        else:
            new_col = 5 - (player_column - col)
            if new_col < 0:
                new_col += 11

        if row > player_row:
            new_row = (3 + row - player_row) % 7
        else:
            new_row = 3 - (player_row - row)
            if new_row < 0:
                new_row += 7
        return new_row, new_col

    food1_row, food1_column = centralize(*row_col(foods[0], configuration.columns))
    food2_row, food2_column = centralize(*row_col(foods[1], configuration.columns))

    food1_row_feat = float(food1_row - 3)/5 if food1_row>=3 else float(food1_row - 3)/5
    food2_row_feat = float(food2_row - 3)/5 if food2_row>=3 else float(food2_row - 3)/5

    food1_col_feat = float(food1_column - 5)/5 if food1_column>=5 else float(food1_column - 5)/5
    food2_col_feat = float(food2_column - 5)/5 if food2_column>=5 else float(food2_column - 5)/5

    # Create the grid
    board = np.zeros([7, 11])
    # Add food to board
    board[food1_row, food1_column] = 15
    board[food2_row, food2_column] = 15

    for geese_id, geese in enumerate(observation.geese):
        nb_blocks = len(geese)
        if nb_blocks > 0:
            for i, pix in enumerate(geese[::-1]):
                if ((i+1) == nb_blocks): #This is the head
                    idx = last_actions_dict[last_actions[geese_id]] #head
                else:
                    idx = (i+5) if (i+5)<15 else 14
                row, col = centralize(*row_col(pix, configuration.columns))
                board[row, col] = idx
            
    return board, len(player_goose), longuest_opponent, food1_row_feat, food1_col_feat, food2_row_feat, food2_col_feat

In [ ]:
class RuleBasedAgent:
    """
    Rule based agent - 
    We will use this rule-based agent to collect state-space data and the actions to take.
    An initial neural network will be trained to learn this rule-based policy.
    The neural network will then be improved using RL methods.
    """
    def __init__(self):
        self.last_action = None
        self.last_heads_positions = []
        self.stateSpace = None
        
    def getStateSpace(self, obs_dict, config_dict):
        heads_positions = geese_heads(obs_dict, config_dict)
        last_actions = get_last_actions(self.last_heads_positions, heads_positions)
        
        board, player_goose_len, longuest_opponent, food1_row_feat, food1_col_feat, food2_row_feat, food2_col_feat = central_state_space(obs_dict, config_dict, last_actions)
        
        cur_obs = {}
        cur_obs['food1_col'] = food1_col_feat
        cur_obs['food2_col'] = food2_col_feat
        cur_obs['food1_row'] = food1_row_feat
        cur_obs['food2_row'] = food2_row_feat
        cur_obs['goose_size'] = player_goose_len
        cur_obs['longuest_opponent'] = longuest_opponent
        cur_obs['board'] = board
        cur_obs['hunger'] = -1 + (float(obs_dict['step']%40)/20)
        cur_obs['step'] = (float(obs_dict['step'])/100) - 1
        
        return cur_obs, heads_positions, last_actions
    def __call__(self, obs_dict, config_dict):
        cur_obs, heads_positions, last_actions = self.getStateSpace(obs_dict, config_dict)
        
        food1_col_feat = cur_obs['food1_col'] 
        food2_col_feat = cur_obs['food2_col'] 
        food1_row_feat = cur_obs['food1_row']
        food2_row_feat = cur_obs['food2_row']
        player_goose_len = cur_obs['goose_size']
        board = cur_obs['board'] 
        cur_obs['hunger'] = -1 + (float(obs_dict['step']%40)/20)
        cur_obs['step'] = (float(obs_dict['step'])/100) - 1

        self.stateSpace = cur_obs
        
        # Prioritize food that is closer
        if (abs(food1_row_feat) + abs(food1_col_feat)) <= (abs(food2_row_feat) + abs(food2_col_feat)):
            p1_food_row_feat = food1_row_feat
            p1_food_col_feat = food1_col_feat
            p2_food_row_feat = food2_row_feat
            p2_food_col_feat = food2_col_feat
        else:
            p1_food_row_feat = food2_row_feat
            p1_food_col_feat = food2_col_feat
            p2_food_row_feat = food1_row_feat
            p2_food_col_feat = food1_col_feat
            

        action = None
        
        
        action_dict = {}

        # For each possible action, we create a value using the following logic:
            # Is action eligible? If yes, +10 000 points
            # Will the action kill us right away? if no, +1000 points
            # Is there a possibility that any other player 
                # move to that same box at that same step? If no, +100 points
            # Is this action getting us closer to the nearest food? If yes, +10 points
            # Is this action getting us closer to the other food? If yes, +1 points
            
        # We then take the action with the most points (won't kill us and
        # brings us toward food if possible)
        
        hunger_boost = 1
        if player_goose_len == 1:
            if (40 - obs_dict['step']%40) < 6:
                hunger_boost = 10
            elif (40 - obs_dict['step']%40) < 3:
                hunger_boost = 100
                
        action_dict[Action.WEST.name] = 0
        # Is action eligible?
        if (self.last_action is None) | (self.last_action != Action.EAST.name):
            action_dict[Action.WEST.name] += 1E7
        # Will the action kill us right away?
        if (board[3, 4] == 0) | (board[3, 4] == 15):
            action_dict[Action.WEST.name] += 1E6
        # Will the action kill us on the subsequent step?:
        if not((board[2, 4] in list(range(6,15))) & (board[3, 3] in list(range(6,15))) & (board[4, 4] in list(range(6,15)))):
            action_dict[Action.WEST.name] += 1E5
        # Could the action kill us on the subsequent step? - is there a head nearby?
        if not((board[2, 4] in list(range(1,5))) & (board[3, 3] in list(range(1,5))) & (board[4, 4] in list(range(1,5)))):
            action_dict[Action.WEST.name] += 1E4
        # Could the action kill us on the subsequent step? - is there a head further?
        if  (not ((board[2, 3] in list(range(1,5))) | (board[3, 2] in list(range(1,5))) | (board[4, 3] in list(range(1,5))))):
            action_dict[Action.WEST.name] += 1E3
        # Is there a possibility that any other player 
        # move to that same box at that same step?
        if (board[3, 3] in [0, 1, 15]) & (board[4, 4] in [0, 4, 15]) & (board[2, 4] in [0, 3, 15]):
            action_dict[Action.WEST.name] += 1E2
        # Is this action getting us closer to the nearest food?
        if p1_food_col_feat < 0:
            action_dict[Action.WEST.name] += 1E1 * hunger_boost
        # Is this action getting us closer to the other food?
        if p2_food_col_feat < 0:
            action_dict[Action.WEST.name] += 1E0 * hunger_boost
            
        action_dict[Action.EAST.name] = 0
        if (self.last_action is None) | (self.last_action != Action.WEST.name):
            action_dict[Action.EAST.name] += 1E7
        if (board[3, 6] == 0) | (board[3, 6] == 15):
            action_dict[Action.EAST.name] += 1E6
        # Will the action kill us on the subsequent step?:
        if not((board[2, 6] in list(range(6,15))) & (board[3, 7] in list(range(6,15))) & (board[4, 6] in list(range(6,15)))):
            action_dict[Action.EAST.name] += 1E5
        # Could the action kill us on the subsequent step? - is there a head nearby?
        if not((board[2, 6] in list(range(1,5))) & (board[3, 7] in list(range(1,5))) & (board[4, 6] in list(range(1,5)))):
            action_dict[Action.EAST.name] += 1E4
        # Could the action kill us on the subsequent step? - is there a head further?
        if  (not ((board[2, 7] in list(range(1,5))) | (board[3, 8] in list(range(1,5))) | (board[4, 7] in list(range(1,5))))):
            action_dict[Action.EAST.name] += 1E3
        if (board[3, 7] in [0, 2, 15]) & (board[4, 6] in [0, 4, 15]) & (board[2, 6] in [0, 3, 15]):
            action_dict[Action.EAST.name] += 1E2
        if p1_food_col_feat > 0:
            action_dict[Action.EAST.name] += 1E1 * hunger_boost
        if p2_food_col_feat > 0: 
            action_dict[Action.EAST.name] += 1E0 * hunger_boost
            

        action_dict[Action.NORTH.name] = 0
        if (self.last_action is None) | (self.last_action != Action.SOUTH.name):
            action_dict[Action.NORTH.name] += 1E7
        if (board[2, 5] == 0) | (board[2, 5] == 15):
            action_dict[Action.NORTH.name] += 1E6
        # Will the action kill us on the subsequent step?:
        if not((board[2, 4] in list(range(6,15))) & (board[2, 6] in list(range(6,15))) & (board[1, 5] in list(range(6,15)))):
            action_dict[Action.NORTH.name] += 1E5
        # Will the action kill us on the subsequent step?  - is there a head nearby?
        if not((board[2, 4] in list(range(1,5))) & (board[2, 6] in list(range(1,5))) & (board[1, 5] in list(range(1,5)))):
            action_dict[Action.NORTH.name] += 1E4
        # Could the action kill us on the subsequent step? - is there a head further?
        if  (not ((board[1, 4] in list(range(1,5))) | (board[0, 5] in list(range(1,5))) | (board[1, 6] in list(range(1,5))))):
            action_dict[Action.NORTH.name] += 1E3
        if (board[1, 5] in [0, 3, 15]) & (board[2, 4] in [0, 1, 15]) & (board[2, 6] in [0, 2, 15]):
            action_dict[Action.NORTH.name] += 1E2
        if p1_food_row_feat < 0:
            action_dict[Action.NORTH.name] += 1E1 * hunger_boost
        if p2_food_row_feat < 0:
            action_dict[Action.NORTH.name] += 1E0 * hunger_boost
            
        action_dict[Action.SOUTH.name] = 0
        if (self.last_action is None) | (self.last_action != Action.NORTH.name):
            action_dict[Action.SOUTH.name] += 1E7
        if (board[4, 5] == 0) | (board[4, 5] == 15):
            action_dict[Action.SOUTH.name] += 1E6
        # Will the action kill us on the subsequent step?:
        if not((board[4, 4] in list(range(6,15))) & (board[4, 6] in list(range(6,15))) & (board[5, 5] in list(range(6,15)))):
            action_dict[Action.SOUTH.name] += 1E5
        # Will the action kill us on the subsequent step? - is there a head nearby?
        if not((board[4, 4] in list(range(1,5))) & (board[4, 6] in list(range(1,5))) & (board[5, 5] in list(range(1,5)))):
            action_dict[Action.SOUTH.name] += 1E4
        # Could the action kill us on the subsequent step? - is there a head further?
        if  (not ((board[5, 4] in list(range(1,5))) | (board[6, 5] in list(range(1,5))) | (board[5, 6] in list(range(1,5))))):
            action_dict[Action.SOUTH.name] += 1E3
        if (board[5, 5] in [0, 4, 15]) & (board[4, 4] in [0, 1, 15]) & (board[4, 6] in [0, 2, 15]):
            action_dict[Action.SOUTH.name] += 1E2
        if p1_food_row_feat > 0:
            action_dict[Action.SOUTH.name] += 1E1 * hunger_boost
        if p2_food_row_feat > 0:
            action_dict[Action.SOUTH.name] += 1E0 * hunger_boost
        
        actions = [Action.SOUTH.name, Action.NORTH.name, Action.EAST.name, Action.WEST.name]
        values = [action_dict[action] for action in actions]
        
        action = actions[np.argmax(values)]

        
        self.last_action = action
        self.last_heads_positions = heads_positions
        return action

In [ ]:
from random import choice
from copy import deepcopy
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, \
                                                                row_col, adjacent_positions, translate, min_distance
class GreedyAgent:
    def __init__(self):
        
        self.last_action = None
        self.observations = []

    def __call__(self, observation: Observation, configuration: Configuration):
        self.configuration = configuration
        
        board = np.zeros(self.configuration.rows*self.configuration.columns)
        board_shape = (self.configuration.rows, self.configuration.columns)
        
        board_heads = deepcopy(board)
        board_bodies = deepcopy(board)
        board_rewards = deepcopy(board)
        
        
        rows, columns = self.configuration.rows, self.configuration.columns

        food = observation.food
        geese = observation.geese
        
        
        opponents = [
            goose
            for index, goose in enumerate(geese)
            if index != observation.index and len(goose) > 0
        ]

        
        opponent_heads = [opponent[0] for opponent in opponents]
        # Don't move adjacent to any heads
        head_adjacent_positions = {
            opponent_head_adjacent
            for opponent_head in opponent_heads
            for opponent_head_adjacent in adjacent_positions(opponent_head, columns, rows)
        }
        
        tail_adjacent_positions ={
            opponent_tail_adjacent
            for opponent in opponents
            for opponent_tail in [opponent[-1]]
            for opponent_tail_adjacent in adjacent_positions(opponent_tail, columns, rows)
        }
        # Don't move into any bodies
        #bodies, heads = [position for goose in geese for position in goose]
        
        heads = [i[0] for i in geese if len(i)>1]
        bodies = [item for sublist in geese for item in sublist]
        
        board_bodies[list(bodies)] = 1
        board_heads[heads] = 1

        # Move to the closest food
        position = geese[observation.index][0]
        actions = {
            action: min_distance(new_position, food, columns)
            for action in Action
            for new_position in [translate(position, action, columns, rows)]
            if (
                new_position not in head_adjacent_positions and
                new_position not in bodies and
                (self.last_action is None or action != self.last_action.opposite())
            )
        }

        action = min(actions, key=actions.get) if any(actions) else choice([action for action in Action])
        
        
        cur_obs = {}
        cur_obs['head_adjacent_positions'] = head_adjacent_positions
        cur_obs['bodies'] = bodies
        cur_obs['board_bodies'] = board_bodies.reshape(board_shape)
        cur_obs['board_heads'] = board_heads.reshape(board_shape)
        cur_obs['tails'] = tail_adjacent_positions
        cur_obs['actions'] = actions
        cur_obs['action'] = action
        cur_obs['last_action'] = self.last_action
#         cur_obs['goose_size'] = player_goose_len
#         cur_obs['board'] = board
        cur_obs['cur_action'] = action
        self.observations.append(cur_obs)
        
        self.last_action = action
        return action.name


cached_greedy_agents = {}


def greedy_agent(obs, config):
    index = obs["index"]
    if index not in cached_greedy_agents:
        cached_greedy_agents[index] = GreedyAgent(Configuration(config))
    return cached_greedy_agents[index](Observation(obs))

In [ ]:
from kaggle_environments import evaluate, make, utils

# Setup a hungry_geese environment.
env = make("hungry_geese", debug = True)
my_agent = RuleBasedAgent()
env.run([my_agent] + ["greedy" for i in range(7)])
env.render(mode="ipython", width=600, height=650)

In [ ]:
my_agent.stateSpace

In [7]:
actions_list = np.array(['EAST',
                        'WEST',
                        'SOUTH',
                        'NORTH'])
def action_to_target(action):
    pos = np.argmax(actions_list == action)
    target = np.zeros(4)
    target[pos] = 1
    return target

def target_to_action(target):
    pos = np.argmax(target)
    return actions_list[pos]

def pred_to_action(pred):
    pos = np.random.multinomial(1, pred)
    return actions_list[pos]

In [8]:
def add_numerical(steps):
    numerical = ['goose_size',
                 'longuest_opponent',
                 'hunger',
                 'step']
    food_position = ['food1_col',
                     'food2_col',
                     'food1_row',
                     'food2_row']
    for step in steps:
        numerical_vector = np.zeros(len(numerical)+1)
        for i, nm in enumerate(numerical):
            if nm in ['goose_size', 'longuest_opponent']:
                numerical_vector[i] = (float(step['cur_state'][nm])-8)/16
            else:
                numerical_vector[i] = step['cur_state'][nm]
        food_position_vector = np.zeros(len(food_position))
        for i, nm in enumerate(food_position):
            food_position_vector[i] = step['cur_state'][nm]
        numerical_vector[len(numerical)] = float(step['cur_state']['goose_size'] - step['cur_state']['longuest_opponent'])/10
        step['numerical'] = numerical_vector
        step['food_position_vector'] = food_position_vector
    return None

In [9]:
def add_embeddings(steps):
    numerical = ['food1_col',
                'food2_col',
                'food1_row',
                'food2_row',
                'goose_size',
                'hunger',
                'step']
    for step in steps:
        #vector = np.zeros(7*11, dtype=int)
        vector = []
        board = step['cur_state']['board']
        for row in range(7):
            for col in range(11):
               #vector[11*row + col] =  np.array(board[row][col], dtype=int)
               vector.append(int(board[row][col]))
        step['embeddings'] = vector
    return None

In [10]:
def add_state_value(discount, steps):
    steps_back = steps[::-1]
    v_prime = 0
    for step in steps_back:
        v = step['reward'] + discount*v_prime
        v_prime = v
        step['v'] = v
    return None

In [11]:
def add_next_state(steps):
    nb_steps = len(steps)
    for i, step in enumerate(steps):
        if step['status'] == 'ACTIVE':
            step['next_embeddings'] = steps[i+1]['embeddings']
            step['next_food_position_vector'] = steps[i+1]['food_position_vector']
            step['next_numerical'] = steps[i+1]['numerical']
        else:
            step['next_embeddings'] = None
            step['next_food_position_vector'] = None
            step['next_numerical'] = None

In [12]:
def process(discount, episodes):
    for episode in episodes:
        add_embeddings(episode)
        add_numerical(episode)
        add_state_value(discount, episode)
        add_next_state(episode)
    return None

In [16]:
def training_data(episodes):
    targets = []
    next_numerical = []
    next_embeddings = []
    reward = []
    done = []
    v = []
    actions = []
    numerical = []
    embeddings = []
    for episode in episodes:
        for step in episode:
            action = step['action']
            target = action_to_target(action)
            targets.append(target)
            num = step['numerical']
            emb = step['embeddings']
            next_numerical = step['next_numerical']
            next_embeddings = step['next_embeddings']
            numerical.append(num)
            embeddings.append(emb)
            actions.append(action)
            v.append(step['v'])
            done.append(step['done'])
            reward.append(step['reward'])

    target_reshaped = np.array(targets).reshape(-1, 4)
    e = [np.array(embeddings)[:, i].reshape(-1, 1) for i in range(7*11)]
    n = [np.array(numerical)[:, i].reshape(-1, 1) for i in range(5)]
    train = n+e

    if not done:
        e_next = [np.array(next_embeddings)[:, i].reshape(-1, 1) for i in range(7*11)]
        n_next = [np.array(next_numerical)[:, i].reshape(-1, 1) for i in range(5)]
        train_next = n_next+e_next
    else:
        train_next = None

    training_dict = {'state': train,
                     'action': action,
                     'next_state': train_next,
                     'y': target_reshaped,
                     'reward': reward,
                     'v': v,
                     'done': done}
    return training_dict

In [21]:
step_reward = 0
dying_reward = 0
winning_reward = 1
step_200_reward = lambda my_goose, longuest_opponent: winning_reward if my_goose > longuest_opponent else 0
win_game_reward = lambda step, my_goose, longuest_opponent: winning_reward #max((200-step), winning_reward)

discount = 1

nb_opponents = 1

steps_per_ep = 200
num_episodes = 10000


env = make("hungry_geese", debug=True)
config = env.configuration

In [ ]:
import pickle
for it in range(100):
    print(f'starting iteration {it}')
    name = f'it_{it}.pkl'
    episodes = []
    for ep in range(num_episodes):
        print('episode number: ', ep)
        steps = []
        my_agent = RuleBasedAgent()
        agents =  [my_agent] + [(RuleBasedAgent() if np.random.rand()<0.7 else GreedyAgent()) for _ in range(nb_opponents)]
        state_dict = env.reset(num_agents=nb_opponents + 1)[0]
        observation = state_dict['observation']
        my_goose_ind = observation['index']

        reward = state_dict['reward']
        action = state_dict['action']



        done = False
        for step in range(1, steps_per_ep):
            actions = []

            for i, agent in enumerate(agents):
                obs = deepcopy(observation)
                obs['index'] = i
                action = agent(obs, config)
                actions.append(action)

            state_dict = env.step(actions)[0]
            observation = state_dict['observation']
            my_goose_ind = observation['index']

            my_goose_length = len(observation['geese'][my_goose_ind])

            longuest_opponent=0
            for i, goose in enumerate(observation.geese):
                if i != my_goose_ind:
                    opponent_length = len(goose)
                    if opponent_length > longuest_opponent:
                        longuest_opponent = opponent_length

            #new_state, _, _ = agent.getStateSpace(observation, config)

            #reward = state_dict['reward']
            action = state_dict['action']
            status = state_dict['status']

            if status != "ACTIVE":
                done = True

            # Check if my goose died
            if my_goose_length == 0:
                done = True
                reward = dying_reward
            elif (step+1) == steps_per_ep:
                reward = step_200_reward(my_goose_length, longuest_opponent)
                done = True
            elif status != "ACTIVE":
                reward = win_game_reward(step, my_goose_length, longuest_opponent)
            else:
                reward = step_reward

            steps.append({'cur_state': my_agent.stateSpace,
                                    'action': action,
                                    'reward': reward,
                                    'new_state': '',#new_state,
                                    'status': status,
                                    'done': done})
            if done:
#                 print('Done, Step: ', step+1)
#                 print('status, ', status)
                break

            if step%50 == 0:
                pass
                #print(f'We survived {step+1} steps')
        episodes.append(steps)
    process(discount, episodes)
    train_data = training_data(episodes)
    with open(f'../data/{name}', 'wb') as f:
        pickle.dump(train_data, f)

starting iteration 0
episode number:  0
Goose Collision: EAST
episode number:  1
Body Hit: (1, <Action.EAST: 2>, 11, [21, 32, 31, 20, 9, 10, 76, 65, 64, 53, 54, 43, 33, 22, 11])
episode number:  2
Goose Collision: NORTH
episode number:  3
Goose Collision: EAST
episode number:  4
Goose Starved: Action.EAST
Goose Starved: Action.EAST
episode number:  5
episode number:  6
Body Hit: (1, <Action.NORTH: 1>, 12, [23, 34, 45, 56, 67, 1, 12, 11, 22, 33, 43, 42, 41])
episode number:  7
Goose Collision: EAST
episode number:  8
Goose Collision: EAST
episode number:  9
Goose Collision: NORTH
Goose Collision: WEST
episode number:  10
Goose Collision: NORTH
episode number:  11
Goose Collision: EAST
episode number:  12
Goose Collision: WEST
episode number:  13
episode number:  14
Body Hit: (0, <Action.NORTH: 1>, 48, [59, 70, 71, 60, 49, 48, 47, 58, 69, 68, 67, 66])
episode number:  15
episode number:  16
Goose Starved: Action.NORTH
episode number:  17
Goose Collision: EAST
episode number:  18
Goose Co

episode number:  154
Goose Collision: EAST
episode number:  155
Body Hit: (1, <Action.NORTH: 1>, 15, [26, 27, 28, 17, 16, 15, 14, 25, 36, 37, 38, 49])
episode number:  156
Body Hit: (1, <Action.WEST: 4>, 57, [58, 47, 36, 25, 14, 3, 69, 68, 57, 46, 35, 24, 13])
episode number:  157
Goose Collision: SOUTH
episode number:  158
Goose Collision: SOUTH
episode number:  159
Body Hit: (0, <Action.WEST: 4>, 11, [12, 23, 34, 35, 36, 25, 24, 13, 2, 1, 0, 11, 22, 33, 44, 55, 56])
episode number:  160
Goose Collision: EAST
episode number:  161
Goose Starved: Action.SOUTH
episode number:  162
Goose Collision: EAST
Goose Collision: NORTH
episode number:  163
Body Hit: (0, <Action.EAST: 2>, 36, [35, 34, 33, 44, 45, 46, 47, 36, 25, 24, 23, 12])
episode number:  164
Goose Collision: EAST
episode number:  165
Goose Collision: EAST
episode number:  166
Goose Collision: WEST
episode number:  167
Goose Collision: EAST
episode number:  168
Goose Collision: NORTH
episode number:  169
Body Hit: (0, <Action.WES

Goose Collision: SOUTH
episode number:  309
episode number:  310
Body Hit: (0, <Action.SOUTH: 3>, 63, [52, 41, 42, 53, 64, 63, 74, 8, 19, 30, 29, 40])
episode number:  311
Goose Collision: EAST
episode number:  312
Goose Collision: SOUTH
episode number:  313
Goose Collision: EAST
episode number:  314
episode number:  315
Goose Collision: SOUTH
episode number:  316
Goose Collision: SOUTH
Goose Collision: EAST
episode number:  317
episode number:  318
Goose Starved: Action.EAST
episode number:  319
Goose Starved: Action.WEST
episode number:  320
Goose Collision: WEST
episode number:  321
Goose Collision: NORTH
episode number:  322
episode number:  323
Goose Collision: SOUTH
Goose Collision: NORTH
episode number:  324
Body Hit: (1, <Action.WEST: 4>, 43, [33, 22, 11, 0, 10, 21, 32, 43])
episode number:  325
Goose Collision: SOUTH
episode number:  326
Goose Collision: SOUTH
episode number:  327
episode number:  328
Body Hit: (0, <Action.WEST: 4>, 59, [60, 61, 62, 63, 52, 51, 50, 49, 48, 59,

episode number:  466
Body Hit: (0, <Action.NORTH: 1>, 37, [48, 59, 58, 47, 36, 37, 26, 15, 4, 70, 69, 68, 67, 56, 57, 46, 35, 34, 33])
episode number:  467
Goose Collision: NORTH
episode number:  468
Goose Starved: Action.EAST
episode number:  469
Goose Collision: NORTH
episode number:  470
Goose Collision: EAST
episode number:  471
Goose Collision: SOUTH
episode number:  472
episode number:  473
Goose Collision: WEST
Goose Collision: SOUTH
episode number:  474
episode number:  475
Body Hit: (1, <Action.SOUTH: 3>, 38, [27, 28, 39, 38, 37, 26, 15, 14, 3, 2, 1, 0])
episode number:  476
episode number:  477
Goose Starved: Action.NORTH
episode number:  478
Goose Starved: Action.NORTH
Goose Starved: Action.NORTH
episode number:  479
Body Hit: (1, <Action.NORTH: 1>, 14, [25, 24, 13, 14, 15, 26, 27, 28])
episode number:  480
Goose Collision: NORTH
Goose Collision: WEST
episode number:  481
Goose Collision: NORTH
episode number:  482
Goose Starved: Action.SOUTH
episode number:  483
Goose Colli

Goose Starved: Action.WEST
episode number:  617
Goose Starved: Action.NORTH
Goose Starved: Action.NORTH
episode number:  618
Goose Collision: NORTH
episode number:  619
Goose Collision: NORTH
episode number:  620
Goose Collision: SOUTH
episode number:  621
Body Hit: (1, <Action.SOUTH: 3>, 36, [25, 14, 13, 24, 35, 36, 37, 26, 15])
episode number:  622
Goose Collision: NORTH
episode number:  623
Body Hit: (1, <Action.NORTH: 1>, 61, [72, 71, 60, 61, 50, 51, 52, 53, 54])
episode number:  624
Goose Collision: WEST
episode number:  625
Body Hit: (1, <Action.SOUTH: 3>, 36, [25, 14, 3, 4, 15, 26, 37, 36, 35, 24])
episode number:  626
Goose Collision: NORTH
Goose Collision: EAST
episode number:  627
episode number:  628
Body Hit: (1, <Action.WEST: 4>, 70, [71, 5, 4, 70, 69, 3, 2, 1])
episode number:  629
Goose Collision: NORTH
Goose Collision: WEST
episode number:  630
Goose Starved: Action.WEST
episode number:  631
Goose Collision: NORTH
episode number:  632
Goose Starved: Action.SOUTH
episode

...

Goose Collision: NORTH
episode number:  9235
Goose Collision: EAST
episode number:  9236
Body Hit: (0, <Action.SOUTH: 3>, 28, [17, 6, 72, 61, 50, 39, 28, 29, 18, 7, 73])
episode number:  9237
Goose Collision: WEST
Goose Collision: NORTH
episode number:  9238
Goose Collision: NORTH
episode number:  9239
Body Hit: (1, <Action.NORTH: 1>, 25, [36, 47, 58, 69, 3, 14, 25, 26])
episode number:  9240
Goose Collision: SOUTH
episode number:  9241
episode number:  9242
Goose Collision: EAST
episode number:  9243
Body Hit: (0, <Action.SOUTH: 3>, 62, [51, 40, 41, 30, 29, 18, 17, 16, 27, 38, 39, 50, 61, 62, 63, 52])
episode number:  9244
Goose Starved: Action.SOUTH
episode number:  9245
Goose Collision: NORTH
episode number:  9246
Body Hit: (1, <Action.NORTH: 1>, 36, [47, 46, 35, 36, 37, 48, 49])
episode number:  9247
Body Hit: (1, <Action.NORTH: 1>, 1, [12, 13, 2, 1, 67, 66, 55])
episode number:  9248
Goose Collision: NORTH
Goose Collision: EAST
episode number:  9249
Body Hit: (1, <Action.NORTH: 1>

Body Hit: (0, <Action.SOUTH: 3>, 39, [28, 29, 30, 19, 18, 17, 16, 27, 38, 39, 40, 51, 62])
episode number:  9383
Goose Collision: EAST
episode number:  9384
Body Hit: (0, <Action.SOUTH: 3>, 45, [34, 33, 43, 54, 44, 45, 46, 35, 24, 23, 22, 11, 0, 1, 67, 68, 69, 70, 4, 15])
episode number:  9385
episode number:  9386
episode number:  9387
Goose Starved: Action.EAST
Goose Starved: Action.WEST
episode number:  9388
Goose Collision: WEST
episode number:  9389
Goose Collision: SOUTH
Goose Collision: NORTH
episode number:  9390
Goose Collision: SOUTH
episode number:  9391
Goose Starved: Action.EAST
Goose Starved: Action.EAST
episode number:  9392
episode number:  9393
Goose Starved: Action.NORTH
Goose Starved: Action.SOUTH
episode number:  9394
Goose Collision: SOUTH
Goose Collision: WEST
episode number:  9395
Goose Collision: EAST
episode number:  9396
Body Hit: (0, <Action.EAST: 2>, 1, [0, 66, 76, 10, 21, 11, 12, 1, 67, 56, 55, 65])
episode number:  9397
Goose Collision: NORTH
Goose Collisi

Goose Collision: EAST
episode number:  9560
Goose Collision: EAST
episode number:  9561
Goose Collision: EAST
episode number:  9562
Goose Collision: NORTH
episode number:  9563
Goose Starved: Action.SOUTH
Goose Starved: Action.NORTH
episode number:  9564
Goose Collision: WEST
episode number:  9565
episode number:  9566
Goose Collision: EAST
episode number:  9567
Goose Collision: NORTH
episode number:  9568
episode number:  9569
Goose Collision: WEST
Goose Collision: SOUTH
episode number:  9570
Goose Collision: WEST
episode number:  9571
Goose Collision: SOUTH
episode number:  9572
Goose Starved: Action.WEST
episode number:  9573
Goose Collision: NORTH
episode number:  9574
episode number:  9575
Body Hit: (0, <Action.EAST: 2>, 66, [76, 65, 55, 66, 0, 10, 9, 75, 64, 53, 54, 44, 45])
episode number:  9576
Goose Collision: SOUTH
Goose Collision: WEST
episode number:  9577
Goose Collision: SOUTH
episode number:  9578
episode number:  9579
Goose Collision: EAST
episode number:  9580
Goose St

episode number:  9711
Goose Collision: NORTH
episode number:  9712
Body Hit: (1, <Action.NORTH: 1>, 40, [51, 52, 53, 42, 41, 40, 29, 18, 19, 20, 21, 11, 12, 13])
episode number:  9713
Goose Collision: WEST
episode number:  9714
Goose Collision: NORTH
episode number:  9715
Body Hit: (1, <Action.EAST: 2>, 20, [19, 8, 74, 63, 52, 41, 30, 31, 20, 9, 75, 64, 53, 54])
episode number:  9716
episode number:  9717
Goose Starved: Action.NORTH
Goose Starved: Action.NORTH
episode number:  9718
Goose Collision: EAST
episode number:  9719
Goose Collision: SOUTH
episode number:  9720
Goose Collision: WEST
Goose Collision: EAST
episode number:  9721
Goose Starved: Action.WEST
Goose Starved: Action.WEST
episode number:  9722
Body Hit: (0, <Action.WEST: 4>, 47, [48, 49, 50, 51, 40, 39, 38, 37, 36, 47, 46, 45, 44, 33, 43])
episode number:  9723
Goose Collision: WEST
episode number:  9724
Body Hit: (0, <Action.SOUTH: 3>, 12, [1, 0, 66, 55, 56, 67, 68, 2, 13, 12, 11])
episode number:  9725
Goose Collision:

Goose Collision: NORTH
episode number:  9861
episode number:  9862
Goose Starved: Action.SOUTH
episode number:  9863
Body Hit: (1, <Action.SOUTH: 3>, 2, [68, 67, 1, 2, 13, 14, 25, 26, 37, 48, 49, 60, 61])
episode number:  9864
Goose Collision: SOUTH
episode number:  9865
episode number:  9866
Goose Collision: EAST
episode number:  9867
Goose Starved: Action.NORTH
Goose Starved: Action.NORTH
episode number:  9868
Body Hit: (1, <Action.WEST: 4>, 32, [22, 11, 0, 66, 76, 65, 54, 44, 33, 43, 32, 21, 20, 9, 8, 7, 18, 19, 30, 41, 52])
episode number:  9869
Goose Collision: NORTH
episode number:  9870
Goose Collision: NORTH
Goose Collision: WEST
episode number:  9871
Goose Collision: NORTH
Goose Collision: EAST
episode number:  9872
Goose Collision: NORTH
Goose Collision: WEST
episode number:  9873
Body Hit: (0, <Action.NORTH: 1>, 15, [26, 25, 24, 13, 2, 3, 4, 15, 16, 27, 38])
episode number:  9874
Goose Collision: SOUTH
episode number:  9875
Goose Collision: NORTH
episode number:  9876
episod

In [20]:
train_data['y']

array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],


In [688]:
episodes[0]

[{'cur_state': {'food1_col': -0.2,
   'food2_col': -0.6,
   'food1_row': -0.2,
   'food2_row': -0.2,
   'goose_size': 1,
   'longuest_opponent': 1,
   'board': array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0., 16.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0., 15.,  0., 15.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0., 16.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]),
   'hunger': -1.0,
   'step': -1.0},
  'action': 'NORTH',
  'reward': 0,
  'new_state': '',
  'status': 'ACTIVE',
  'done': False,
  'embeddings': [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   16,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   15,
   0,
   15,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
  

### Let's create a network to predict action to take based on state

In [589]:
import os
from math import ceil
import numpy as np
from tensorflow.keras.layers import Dense,Input, Embedding, concatenate,\
    Flatten, Average, Dropout, BatchNormalization, Activation
from tensorflow.keras import Sequential, Model
from tensorflow import config, distribute
import tensorflow as tf


In [590]:
tf.compat.v1.reset_default_graph()

In [591]:
num_layers = []
for _ in range(5):
    num_layers.append(Input(shape=1))

In [592]:
emb_layers = []
for i in range(7*11):
    m = Sequential()
    embedding._name = f'embeddings_{i}'
    m.add(embedding)
    m.add(Flatten(name=f'flat_embeddings-{i}'))
    emb_layers.append(m)

In [593]:
inputs = num_layers + [inp.input for inp in emb_layers]
outputs = num_layers + [inp.output for inp in emb_layers]

In [594]:
c = concatenate(outputs)
model = Dense(100, activation='elu')(c)
model = Dense(50, activation='elu')(model)
model = Dense(20, activation='elu')(model)
model = Dense(10, activation='elu')(model)
model = Dense(4, activation='softmax')(model)

In [595]:
m = Model(inputs, model)
m.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [597]:
m.predict([x.reshape(-1,1) for x in np.array([[16,5,10]] * (5+11*7))])

array([[0.89772683, 0.02073192, 0.01104983, 0.07049137],
       [0.62568307, 0.07258962, 0.15280053, 0.1489268 ],
       [0.8255264 , 0.03592074, 0.04352432, 0.09502851]], dtype=float32)

In [655]:
len([x.reshape(-1,1) for x in np.array([[16,5,10]] * (5+11*7))])

82

In [654]:
[x.reshape(-1,1) for x in np.array([[16,5,10]] * (5+11*7))]

[array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
    

In [598]:
[x.reshape(-1,1) for x in np.array([[16,5,10]] * (5+11*7))]

[array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
        [10]]),
 array([[16],
        [ 5],
    

In [686]:
episodes[0]

[{'cur_state': {'food1_col': -0.2,
   'food2_col': -0.6,
   'food1_row': -0.2,
   'food2_row': -0.2,
   'goose_size': 1,
   'longuest_opponent': 1,
   'board': array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0., 16.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0., 15.,  0., 15.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0., 16.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]),
   'hunger': -1.0,
   'step': -1.0},
  'action': 'NORTH',
  'reward': 0,
  'new_state': '',
  'status': 'ACTIVE',
  'done': False},
 {'cur_state': {'food1_col': -0.6,
   'food2_col': 0.4,
   'food1_row': 0.0,
   'food2_row': 0.0,
   'goose_size': 1,
   'longuest_opponent': 2,
   'board': array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.

In [ ]:
def training_data(episodes):
    targets = []
    next_numerical = []
    next_embeddings = []
    reward = []
    done = []
    v = []
    actions = []
    numerical = []
    embeddings = []
    for episode in episodes:
        for step in episode:
            action = step['action']
            target = action_to_target(action)
            targets.append(target)
            num = step['numerical']
            emb = step['embeddings']
            next_numerical = step['next_numerical']
            next_embeddings = step['next_embeddings']
            numerical.append(num)
            embeddings.append(emb)
            actions.append(action)
            v.append(step['v'])
            done.append(step['done'])
            reward.append(step['reward'])

    target_reshaped = np.array(targets).reshape(-1, 4)
    e = [np.array(embeddings)[:, i].reshape(-1, 1) for i in range(7*11)]
    n = [np.array(numerical)[:, i].reshape(-1, 1) for i in range(5)]
    train = n+e

    e_next = [np.array(next_embeddings)[:, i].reshape(-1, 1) for i in range(7*11)]
    n_next = [np.array(next_numerical)[:, i].reshape(-1, 1) for i in range(5)]
    train_next = n_next+e_next

    training_dict = {'state': train,
                     'action': action,
                     'next_state': train_next,
                     'y': target_reshaped,
                     'reward': reward,
                     'v': v,
                     'done': done}
    return training_dict

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [ ]:
np.array(targets).reshape(-1, 4)

In [660]:
np.array(embeddings)[:, 0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5,  5,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 15,
        0,  0,  0,  7,  7,  7,  0,  0,  0,  0,  0,  6,  0,  7,  7,  7,  7,
        7,  0,  0,  0,  5,  5,  5,  5,  0,  0,  6,  0,  0,  0,  0, 11,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 14, 13, 11,  9,
        0,  3,  1,  3,  1,  0,  0,  0, 14, 14,  6,  6,  6,  6,  0,  0,  0,
        0, 14,  0,  0,  0,  0,  8, 14, 12,  0, 14, 13, 12])

In [661]:
e = [np.array(embeddings)[:, i].reshape(-1, 1) for i in range(7*11)]

In [666]:
n = [np.array(numerical)[:, i].reshape(-1, 1) for i in range(5)]

In [668]:
train = n+e

In [ ]:
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import roc_auc_score, mean_squared_error

def reset_weights(model):
    session = K.get_session()
    for layer in model.layers:
        if hasattr(layer, 'kernel_initializer'):
            layer.kernel.initializer.run(session=session)
        if hasattr(layer, 'bias_initializer'):
            layer.bias.initializer.run(session=session)

class perf_callback(Callback):
    def __init__(self, data, target_type='classification'):
        self.X = data[0]
        self.y = data[1]
        self.target_type = target_type
    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.X)
        if self.target_type == 'classification':
            perf = roc_auc_score(self.y, y_pred, average='micro')
        elif self.target_type == 'regression':
            perf = mean_squared_error(self.y, y_pred)
        logs['validation'] = perf
        

early_stop = EarlyStopping(patience=self.patience,
                           monitor='validation',
                           mode=mode)

In [676]:
from sklearn.model_selection import train_test_split
y = np.array(targets).reshape(-1, 4)

callbacks = [
    perf_callback((X_val_adj, y_val_adj), 'classification'),
    early_stop]

ModuleNotFoundError: No module named 'sklearn'

In [675]:
m.fit(train,
      np.array(targets).reshape(-1, 4),
      epochs=100,
      batch_size=32,
      callbacks=callbacks)

Epoch 1/100
6/6 [==============================] - 0s 12ms/step - loss: 0.8854 - accuracy: 0.6721
Epoch 2/100
6/6 [==============================] - 0s 20ms/step - loss: 0.8237 - accuracy: 0.6995
Epoch 3/100
6/6 [==============================] - 0s 12ms/step - loss: 0.7459 - accuracy: 0.7541
Epoch 4/100
6/6 [==============================] - 0s 15ms/step - loss: 0.6832 - accuracy: 0.8087
Epoch 5/100
6/6 [==============================] - 0s 19ms/step - loss: 0.6072 - accuracy: 0.8306
Epoch 6/100
6/6 [==============================] - 0s 17ms/step - loss: 0.5510 - accuracy: 0.8251
Epoch 7/100
6/6 [==============================] - 0s 18ms/step - loss: 0.5036 - accuracy: 0.8470
Epoch 8/100
6/6 [==============================] - 0s 17ms/step - loss: 0.4394 - accuracy: 0.8689
Epoch 9/100
6/6 [==============================] - 0s 18ms/step - loss: 0.3847 - accuracy: 0.8798
Epoch 10/100
6/6 [==============================] - 0s 23ms/step - loss: 0.3663 - accuracy: 0.8907
Epoch 11/100
6/6 [=

6/6 [==============================] - 0s 9ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 85/100
6/6 [==============================] - 0s 13ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 86/100
6/6 [==============================] - 0s 18ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 87/100
6/6 [==============================] - 0s 15ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 88/100
6/6 [==============================] - 0s 14ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 89/100
6/6 [==============================] - 0s 14ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 90/100
6/6 [==============================] - 0s 6ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 91/100
6/6 [==============================] - 0s 6ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 92/100
6/6 [==============================] - 0s 16ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 93/100
6/6 [==============================] - 0s 12ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 94/100
6/6 [========

In [612]:
targets

[array([0., 1., 0., 0.]),
 array([0., 1., 0., 0.]),
 array([0., 0., 0., 1.]),
 array([0., 1., 0., 0.]),
 array([0., 0., 0., 1.]),
 array([0., 1., 0., 0.]),
 array([0., 0., 0., 1.]),
 array([1., 0., 0., 0.]),
 array([0., 0., 1., 0.]),
 array([0., 0., 1., 0.]),
 array([1., 0., 0., 0.]),
 array([0., 0., 1., 0.]),
 array([1., 0., 0., 0.]),
 array([0., 0., 0., 1.]),
 array([0., 1., 0., 0.]),
 array([0., 0., 1., 0.]),
 array([0., 1., 0., 0.]),
 array([0., 0., 0., 1.]),
 array([0., 1., 0., 0.]),
 array([0., 0., 0., 1.]),
 array([0., 1., 0., 0.]),
 array([0., 0., 0., 1.]),
 array([0., 0., 0., 1.]),
 array([1., 0., 0., 0.]),
 array([1., 0., 0., 0.]),
 array([0., 0., 1., 0.]),
 array([0., 0., 1., 0.]),
 array([0., 0., 1., 0.]),
 array([0., 0., 1., 0.]),
 array([0., 0., 1., 0.]),
 array([0., 1., 0., 0.]),
 array([0., 1., 0., 0.]),
 array([0., 0., 0., 1.]),
 array([0., 1., 0., 0.]),
 array([0., 1., 0., 0.]),
 array([0., 1., 0., 0.]),
 array([0., 1., 0., 0.]),
 array([0., 1., 0., 0.]),
 array([0., 